# <a>Gradient Boosting com XGBoost</a>



---

O Let's Data produz conteúdos gratuitos sobre Data Science!

Confira algumas das nossa produções:

* [Livro Digital - Como se tornar um Cientista de Dados?](https://ebook.letsdata.ai/)
* [Inscreva-se no canal no YouTube](https://www.youtube.com/c/LetsDataAI?sub_confirmation=1)
* [Siga-nos no Instagram](https://www.instagram.com/letsdata.ai)
* Ouça nosso Podcast no [Spotify](https://open.spotify.com/show/0VsNN95jsJVRS424eCFDlg?si=EvQ1KBq1SqGVD4ojHrx4eQ), [Google Podcast](https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkcy5idXp6c3Byb3V0LmNvbS8xNzMxMjUyLnJzcw), [Apple Podcasts](https://podcasts.apple.com/us/podcast/lets-data-podcast/id1559072194), [Castbox](https://castbox.fm/vic/1559072194), [Deezer](https://www.deezer.com/show/2770422) e demais players. 
* Também estamos no [LinkedIn](https://linkedin.com/company/lets-data), [Twitter](https://twitter.com/letsdataAI), [Facebook](https://www.facebook.com/letsdataAI/).
* [Pra finalizar, nosso blog no Medium](https://medium.com/lets-data).

---

## Vamos pro conteúdo!

Queridinhos dos Kaggle Grandmasters, os algoritmos de gradient boosting dominam as aplicações práticas quando o assunto é machine learning para dados tabulares. Hoje veremos uma implementação muito utilizada, e mais badalada pelos grandes DS e MLE do mundo: XGBoost. Arquirrival do LightGBM e Catboost, o XGBBoost é tido como a melhor implementaçãod e gradient boosting disponível. Os títulos e prêmios do Kaggle demonstram claramente isso!

Nessa demonstração vamos usar a API do XGBoost para scikit-learn, assim como fizemos no LightGBM. Existe uma API "nativa" que possui objetos e métodos diferentes, mas se estamos acostumados com os velhos fit, predict, cross_val's da vida, por que não utilizar?

Caso queria aprender a API "original" segue o [link](https://xgboost.readthedocs.io/en/stable/get_started.html)

Lembrando que precisamos instalar o pacote, já que não vem com o scikit learn! Vamos nessa!

In [ ]:
#!pip install xgboost
#!pip install notebook pandas seaborn scikit-learn

In [16]:
# Importando o xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

## <a> Carregando os dados para regressão e classificação </a> 

O foco vai ser em como utilizar o XGBoost, mais que nos problemas de regressão/classificação em si

In [2]:
import seaborn as sns

df_iris = sns.load_dataset('iris')

df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df_iris.shape

(150, 5)

In [4]:
df_iris['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
df_iris['species'] = df_iris['species'].map({'setosa':0 , 'versicolor': 1, 'virginica': 2})

df_iris['species'].unique()

array([0, 1, 2], dtype=int64)

In [6]:
df_iris['species'].value_counts()

0    50
1    50
2    50
Name: species, dtype: int64

## <a> Separando as bases </a>

O foco é XGBoost, mas um splitzinho tem que rolar sempre!

In [7]:
# Separando X e y
X_iris = df_iris.drop(['species'], axis=1)
y_iris = df_iris['species']

In [8]:
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(X_iris, y_iris, random_state=42)

In [11]:
X_iris_train.shape, X_iris_test.shape

((112, 4), (38, 4))

In [13]:
len(X_iris_test) / len(X_iris) 

0.25333333333333335

## <a> Criando o objeto com o classificador XGBoost </a>

Veja o quanto é difícil #sqn

In [17]:
classificador_xgb = xgb.XGBClassifier()

In [18]:
type(classificador_xgb)

xgboost.sklearn.XGBClassifier

### <a> Compatibilidade com scikit-learn </a>

Olha que massa! O [XGBClassifier](https://xgboost.readthedocs.io/en/stable/python/python_api.html) é uma subclasse de *sklearn.base.ClassifierMixin*, ou seja, é compatível com sklearn. Podemos então utilizar o arcabouço do scikit learn, como métodos de CV, pipelines, etc...

In [21]:
from sklearn.model_selection import cross_val_score

# Funciona com scikit learn! <3
resultado_cv = cross_val_score(classificador_xgb, X_iris_train, y_iris_train)

resultado_cv

array([0.95652174, 0.95652174, 0.86363636, 0.95454545, 0.95454545])

In [26]:
acuracia_cv = resultado_cv.mean()

acuracia_cv

0.9371541501976285

In [27]:
acuracia_cv = 100 * acuracia_cv

acuracia_cv

93.71541501976284

### <a> Pequena tunagem! </a>

O XGBoost permite alteração de vários hiperparâmetros, como learning rate, altura máxima das árvores, quantidade máxima de folhas nas árvores, número de árvores (estimadores), etc...

Nosso foco não é em tunagem de hiperparâmetros, mas vamos mudar somente a altura máxima.

Vamos aproveitar também para mostrar uma característica interessante do XGBBoost. Ele permite alterar a implementação do classificador! Podemos, por exemplo, utilizar random forest (bagging) ao invés de algoritmos de boosting! Vamos ver se melhora o resultado.

Um notebook excelente para entender um montão de hiperparâmetros do XGBoost: [disponível no Kaggle](https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook)

Outro hiperparâmetro importante é a função de erro utilizada para otimização: objective 

Como o padrão para classificação é "binary: logistic" e para regressão "reg:squarederror". Mas quando passamos uma variável target com mais de duas classes ele muda automaticamente para "multi:softprob". Vamos ver!

In [30]:
clf = xgb.XGBClassifier()
clf.objective

'binary:logistic'

In [31]:
clf = xgb.XGBClassifier().fit(X_iris_train, y_iris_train)
clf.objective

'multi:softprob'

Poderíamos também utilizar a multi:softmax, mas esta somente retorna a classe com maior probabilidade, enquanto a softprob entrega as probabilidades de todas as classes para cada predição

In [32]:
classificador_xgb_tunado = xgb.XGBClassifier(max_depth=2)

100 * cross_val_score(classificador_xgb_tunado, X_iris_train, y_iris_train).mean()

92.84584980237153

### <a> Tipos de classificador </a>

Direto da [documentação](https://xgboost.readthedocs.io/en/stable/parameter.html): 

'gbtree', 'gblinear' ou 'dart' (gbtree and dart use tree based models while gblinear uses linear functions.)

In [33]:
classificador_xgb_dart =  xgb.XGBClassifier(booster='dart')

100 * cross_val_score(classificador_xgb_dart, X_iris_train, y_iris_train).mean()

93.71541501976284

In [34]:
classificador_xgb_gblinear =  xgb.XGBClassifier(booster='gblinear')

100 * cross_val_score(classificador_xgb_gblinear, X_iris_train, y_iris_train).mean()

94.62450592885375

In [35]:
# Capricho de legibilidade
classificador_campeao = classificador_xgb_gblinear

In [36]:
# Com o melhor modelo, podemos utilizar a base toda de treino
classificador_campeao.fit(X_iris_train, y_iris_train)

XGBClassifier(base_score=0.5, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None, gpu_id=-1,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=0,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, random_state=0, reg_alpha=0, ...)

In [37]:
# Podemos realizar a predição da base de teste!
predicoes_iris = classificador_campeao.predict(X_iris_test)

predicoes_iris[:10]

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1], dtype=int64)

In [39]:
type(y_iris_test)

pandas.core.series.Series

In [38]:
y_iris_test

73     1
18     0
118    2
78     1
76     1
31     0
64     1
141    2
68     1
82     1
110    2
12     0
36     0
9      0
19     0
56     1
104    2
69     1
55     1
132    2
29     0
127    2
26     0
128    2
131    2
145    2
108    2
143    2
45     0
30     0
22     0
15     0
65     1
11     0
42     0
146    2
51     1
27     0
Name: species, dtype: int64

In [40]:
# Calculando o número de acertos
(predicoes_iris == y_iris_test).sum()

38

In [41]:
# Mas qual o tamanho da base de teste?
len(y_iris_test)

38

In [42]:
acertos = (predicoes_iris == y_iris_test).sum()
total_observacoes = len(y_iris_test)

acuracia = 100 * acertos / total_observacoes

acuracia

100.0

## <a> Vamos de Regressão? </a>

Bem rapidinho porque é praticamente a mesma coisa, mudando só o objeto de XGBClassifier para XGBRegressor

In [43]:
# Vamos tentar prever o consumo dos carros a partir de suas features
df_mpg = sns.load_dataset('mpg')

df_mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [44]:
df_mpg.shape

(398, 9)

In [45]:
df_mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [46]:
# Leon é preguiçoso e tirou origin e name
X_mpg = df_mpg.drop(['mpg', 'origin', 'name'], axis=1)
y_mpg = df_mpg['mpg']

In [47]:
X_mpg_train, X_mpg_test, y_mpg_train, y_mpg_test = train_test_split(X_mpg, y_mpg, random_state=42)

### <a> E a função objetivo? </a>

Lembram que existe o hiperparametro objective que determina a função de erro a ser utilizada para otimização.

Somente checando o padrão para a regressão.
    

In [48]:
clf = xgb.XGBRegressor()
clf.objective

'reg:squarederror'

In [49]:
# Agora sim! Bem facinho, só mudar de XGBClassifier para XGBRegressor
regressor_xgb = xgb.XGBRegressor()

In [50]:
regressor_xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, ...)

In [51]:
regressor_xgb.objective

'reg:squarederror'

In [52]:
cross_val_score(regressor_xgb, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.1598072730534854

In [53]:
regressor_xgb_tunado = xgb.XGBRegressor(max_depth=2)

cross_val_score(regressor_xgb_tunado, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.1923225740762793

In [54]:
regressor_xgb_gblinear =  xgb.XGBRegressor(booster='gblinear')

cross_val_score(regressor_xgb_gblinear, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-4.233149423803371

In [55]:
regressor_campeao = regressor_xgb

In [57]:
regressor_campeao = xgb.XGBRegressor(eval_metric='rmsle')

regressor_campeao.fit(X_mpg_train,y_mpg_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmsle', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [58]:
predicoes_mpg = regressor_campeao.predict(X_mpg_test)

predicoes_mpg[:10]

array([30.606602, 30.937365, 21.048904, 14.121838, 13.470102, 24.374348,
       29.221304, 11.992093, 16.482681, 15.383792], dtype=float32)

In [59]:
y_mpg_test[:10]

198    33.0
396    28.0
33     19.0
208    13.0
93     14.0
84     27.0
373    24.0
94     13.0
222    17.0
126    21.0
Name: mpg, dtype: float64

In [60]:
from sklearn.metrics import mean_squared_error
import math

mse = mean_squared_error(y_mpg_test, predicoes_mpg)

display(mse)

rmse = math.sqrt(mse)

rmse



10.182896418983752

3.1910650916243863